# Práctica 1: Procesamiento de Imágenes, audio y vídeo

### Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

In [1]:
import cv2 as cv
import numpy as np

#### 1a. Hacer un programa que cargue una imagen y que obtenga y muestre el valor RGB de cada píxel de la imagen sobre el que esté el cursor del ratón.

In [12]:
img = cv.imread('./images/colores.jpg', cv.IMREAD_COLOR)
img_original = img.copy()

def mouse_event(event, x, y, flags, param):
    if event == cv.EVENT_MOUSEMOVE:

        img[:] = img_original.copy()

        bgr = img[y, x]
        text = f'RGB ({bgr[2]}, {bgr[1]}, {bgr[0]})'
        cv.rectangle(img, (x, y + 5), (x + 170, y - 15), (0, 0, 0), -1)
        cv.putText(img, text, (x, y), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv.LINE_AA)
        

    cv.imshow('colores', img)

cv.imshow('colores', img)
cv.setMouseCallback('colores', mouse_event)
cv.waitKey(0)
cv.destroyAllWindows()

### 1b. Hacer un programa para dibujar las siguientes primitivas con el ratón

- Líneas
- Rectángulos
- Círculos
- Especificar el color en RGB de las primitivas
- Definir el grosor de las líneas

In [17]:
img = np.zeros((512, 512, 3), dtype=np.uint8) + 255
img_orig = img.copy()

drawing = False 
mode = 'line'
start_point = (-1, -1)
color = (0, 0, 0)
thickness = 1

def nothing(x):
    pass

def draw_shape(event, x, y, flags, param):
    global start_point, drawing, mode, color, thickness, img_orig

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)

    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            if mode == 'line':
                img_copy = img.copy()
                cv.line(img_copy, start_point, (x, y), color, thickness)
                cv.imshow('image', img_copy)
            elif mode == 'rectangle':
                img_copy = img.copy()
                cv.rectangle(img_copy, start_point, (x, y), color, thickness)
                cv.imshow('image', img_copy)
            elif mode == 'circle':
                img_copy = img.copy()
                radius = int(((x - start_point[0]) * 2 + (y - start_point[1]) * 2) ** 0.5)
                cv.circle(img_copy, start_point, radius, color, thickness)
                cv.imshow('image', img_copy)
    
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        if mode == 'line':
            cv.line(img, start_point, (x, y), color, thickness)
        elif mode == 'rectangle':
            cv.rectangle(img, start_point, (x, y), color, thickness)
        elif mode == 'circle':
            radius = int(((x - start_point[0]) * 2 + (y - start_point[1]) * 2) ** 0.5)
            cv.circle(img, start_point, radius, color, thickness)
        cv.imshow('image', img)

cv.namedWindow('image')
cv.setMouseCallback('image', draw_shape)

cv.createTrackbar('R','image',0,255,nothing)
cv.createTrackbar('G','image',0,255,nothing)
cv.createTrackbar('B','image',0,255,nothing)
cv.createTrackbar('Thickness', 'image', 1, 5, nothing)

def change_color(r, g, b):
    global color
    color = (b, g, r)

def change_thickness(new_thickness):
    global thickness
    thickness = new_thickness

while True:
    cv.imshow('image', img)
    key = cv.waitKey(1) & 0xFF

    change_color(cv.getTrackbarPos('R','image'), cv.getTrackbarPos('G','image'), cv.getTrackbarPos('B','image'))
    change_thickness( cv.getTrackbarPos('Thickness','image'))
    
    if key == ord('m'):
        mode = 'line' if mode == 'circle' else 'rectangle' if mode == 'line' else 'circle'
    elif key == ord('q'):
        break
    elif key == ord('d'):
        img[:] = img_orig

cv.destroyAllWindows()

### 2. Mejoras del programa:

- Permitir crear figuras rellenas (seleccionar a través de la interfaz)
- Poder crear otras primitivas (construir polilíneas y polígonos, elipses…)
- Guardar el dibujo como un vídeo para ver cómo se realizó paso a paso
- Aportación propia


In [54]:
img = np.zeros((1024, 1024, 3), dtype=np.uint8) + 255
img_orig = img.copy()

drawing = False 
mode = 'line'
start_point = (-1, -1)
color = (0, 0, 0)
thickness = 1
filled = False
points = []
actions = []

def nothing(x):
    pass

def draw_shape(event, x, y, flags, param):
    global start_point, drawing, mode, color, thickness, img_orig, points

    # Dibujar la figura al hacer click
    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)
        if mode in ['polyline', 'polygon']:
            points.append((x, y))

    # Dibujar la figura mientras se mueve el mouse
    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            if mode == 'line':
                cv.line(img_copy, start_point, (x, y), color, thickness)
            elif mode == 'rectangle':
                cv.rectangle(img_copy, start_point, (x, y), color, thickness if not filled else -1)
            elif mode == 'circle':
                radius = int(((x - start_point[0]) ** 2 + (y - start_point[1]) ** 2) ** 0.5)
                cv.circle(img_copy, start_point, radius, color, thickness if not filled else -1)
            elif mode == 'ellipse':
                axes = (abs(x - start_point[0]), abs(y - start_point[1]))
                cv.ellipse(img_copy, start_point, axes, 0, 0, 360, color, thickness if not filled else -1)
            elif mode in ['polyline', 'polygon'] and len(points) > 1:
                cv.polylines(img_copy, [np.array(points)], False, color, thickness)
            display_mode(img_copy)
            cv.imshow('image', img_copy)

    # Dibujar la figura al soltar el click
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        if mode == 'line':
            cv.line(img, start_point, (x, y), color, thickness)
        elif mode == 'rectangle':
            cv.rectangle(img, start_point, (x, y), color, thickness if not filled else -1)
        elif mode == 'circle':
            radius = int(((x - start_point[0]) ** 2 + (y - start_point[1]) ** 2) ** 0.5)
            cv.circle(img, start_point, radius, color, thickness if not filled else -1)
        elif mode == 'ellipse':
            axes = (abs(x - start_point[0]), abs(y - start_point[1]))
            cv.ellipse(img, start_point, axes, 0, 0, 360, color, thickness if not filled else -1)
        elif mode in ['polyline', 'polygon']:
            cv.polylines(img, [np.array(points)], False, color, thickness)

        actions.append(img.copy())
        cv.imshow('image', img)


# Crear una ventana y establecer la función de callback para el ratón
cv.namedWindow('image')
cv.setMouseCallback('image', draw_shape)

# Crear trackbars para ajustar el color, grosor y relleno
cv.createTrackbar('R','image',0,255,nothing)
cv.createTrackbar('G','image',0,255,nothing)
cv.createTrackbar('B','image',0,255,nothing)
cv.createTrackbar('Thickness', 'image', 1, 5, nothing)
cv.createTrackbar('Filled', 'image', 0, 1, nothing)

def change_color(r, g, b):
    global color
    color = (b, g, r)

def change_thickness(new_thickness):
    global thickness
    thickness = new_thickness

def change_filled(filled_value):
    global filled
    filled = bool(filled_value)

def display_mode(image):
    font = cv.FONT_HERSHEY_SIMPLEX
    cv.putText(image, f'Mode: {mode}', (10, 30), font, 0.75, (0, 0, 0), 2, cv.LINE_AA)

# Dimensiones de la imagen y creación del video
height, width, _ = img.shape
video = cv.VideoWriter('drawing.mp4', cv.VideoWriter_fourcc(*'MP4V'), 160, (width, height))

# Mostrar la imagen inicial
cv.imshow('image', img)

while True:
    img_with_mode = img.copy()
    display_mode(img_with_mode)
    cv.imshow('image', img_with_mode)
    video.write(img)
    key = cv.waitKey(1) & 0xFF

    # Cambiar color, grosor y relleno basados en los valores de los trackbars
    change_color(cv.getTrackbarPos('R','image'), cv.getTrackbarPos('G','image'), cv.getTrackbarPos('B','image'))
    change_thickness(cv.getTrackbarPos('Thickness','image'))
    change_filled(cv.getTrackbarPos('Filled','image'))

    # Cambiar modo de dibujo con la tecla 'm'
    if key == ord('m'):
        if mode == 'line':
            mode = 'rectangle'
        elif mode == 'rectangle':
            mode = 'circle'
        elif mode == 'circle':
            mode = 'ellipse'
        elif mode == 'ellipse':
            mode = 'polyline'
        elif mode == 'polyline':
            points = []
            mode = 'polygon'
        elif mode == 'polygon':
            mode = 'line'
    
    # Salir del programa con 'q'
    elif key == ord('q'):
        break
    
    # Limpiar la imagen y reiniciar
    elif key == ord('d'):
        img = img_orig.copy()
        cv.imshow('image', img)
        actions = []
    
    # Deshacer la última acción con 'r'
    elif key == ord('r') and actions:
        actions.pop()
        if actions:
            img = actions[-1].copy()
        else:
            img = img_orig.copy()
        cv.imshow('image', img)

    elif key == ord('c'):
        if mode == 'polygon':
            if len(points) > 2:
                cv.polylines(img, [np.array(points)], isClosed=True, color=(0, 0, 255), thickness=thickness)
                points = []
        elif mode == 'polyline':
            points = []
        cv.imshow('image', img)

video.release()
cv.destroyAllWindows()